In [6]:
from heapq import heappush, heappop
from typing import Dict, List, Tuple, Optional


def build_graph() -> Dict[str, Dict[str, int]]:
    edges = [
        ("A", "B", 2),
        ("A", "C", 4),
        ("B", "D", 3),
        ("B", "E", 2),
        ("C", "E", 2),
        ("C", "F", 3),
        ("E", "G", 2),
    ]
    g: Dict[str, Dict[str, int]] = {}
    for u, v, w in edges:
        g.setdefault(u, {})[v] = w
        g.setdefault(v, {})[u] = w
    return g


def astar(
    graph: Dict[str, Dict[str, int]],
    start: str,
    goal: str,
    h: Dict[str, float],
    neighbor_order: Optional[Dict[str, List[str]]] = None,
) -> Tuple[List[str], Dict[str, float], List[str]]:

    def ordered_neighbors(u: str) -> List[str]:
        nbrs = list(graph[u].keys())
        if neighbor_order and u in neighbor_order:
            order = neighbor_order[u]
            seen = set(nbrs)
            ord_nbrs = [x for x in order if x in seen]
            tail = sorted(list(seen - set(ord_nbrs)))
            return ord_nbrs + tail
        return sorted(nbrs)

    g: Dict[str, float] = {start: 0.0}
    parent: Dict[str, Optional[str]] = {start: None}
    expansions: List[str] = [] 
    
    open_heap: List[Tuple[float, float, float, str]] = []
    heappush(open_heap, (g[start] + h[start], h[start], g[start], start))

    closed: set[str] = set()
    in_open: Dict[str, Tuple[float, float, float, str]] = {start: (g[start] + h[start], h[start], g[start], start)}

    while open_heap:
        f_cur, h_cur, g_cur, u = heappop(open_heap)
        if u in closed:
            continue
        if u not in g or g[u] != g_cur:
            continue

        closed.add(u)
        expansions.append(u)

        if u == goal:
            path: List[str] = []
            x = u
            while x is not None:
                path.append(x)
                x = parent[x]
            path.reverse()
            return path, g, expansions

        for v in ordered_neighbors(u):
            cost = graph[u][v]
            tentative_g = g[u] + cost
            if v in closed:
                continue
            if (v not in g) or (tentative_g < g[v]):
                g[v] = tentative_g
                parent[v] = u
                f_v = tentative_g + h[v]
                rec = (f_v, h[v], tentative_g, v)
                heappush(open_heap, rec)
                in_open[v] = rec


    return [], g, expansions


if __name__ == "__main__":
    graph = build_graph()
    h = {
        "A": 6, "B": 5, "C": 4, "D": 3,
        "E": 2, "F": 1, "G": 0
    }

    path, g_costs, expansions = astar(graph, start="A", goal="G", h=h)

    print("A* expansions order:", expansions)        
    print("A* path:", " -> ".join(path))              
    print("Total cost:", g_costs.get("G", float("inf"))) 


A* expansions order: ['A', 'B', 'E', 'G']
A* path: A -> B -> E -> G
Total cost: 6.0


In [7]:
from typing import Dict, List, Tuple, Optional, Set
import copy


Color = str
Var = str
Domains = Dict[Var, List[Color]]
Adj = Dict[Var, Set[Var]]

def csp_topology() -> Tuple[List[Var], Domains, Adj]:

    variables = ["R1", "R2", "R3", "R4"]
    colors = ["Red", "Green", "Blue"]
    domains: Domains = {v: list(colors) for v in variables}
    adj: Adj = {
        "R1": {"R2", "R3", "R4"},
        "R2": {"R1", "R3"},
        "R3": {"R1", "R2", "R4"},
        "R4": {"R1", "R3"},
    }
    return variables, domains, adj


def degree_heuristic(variables: List[Var], adj: Adj) -> Dict[Var, int]:
    return {v: len(adj[v]) for v in variables}


def select_var_MRV_degree(assignment: Dict[Var, Color], domains: Domains, adj: Adj) -> Var:

    candidates = [v for v in domains if v not in assignment]
    deg = degree_heuristic(list(domains.keys()), adj)
    candidates.sort(key=lambda v: (len(domains[v]), -deg[v], v))
    return candidates[0]


def order_values_LCV(var: Var, domains: Domains, adj: Adj) -> List[Color]:

    scores = []
    for val in domains[var]:
        impact = 0
        for n in adj[var]:
            if n in domains and val in domains[n] and len(domains[n]) > 1:
                impact += 1 
        scores.append((impact, val))
    scores.sort(key=lambda x: (x[0], x[1])) 
    return [v for _, v in scores]


def forward_check(var: Var, val: Color, domains: Domains, adj: Adj) -> Tuple[Domains, bool, List[Tuple[Var, Color]]]:

    new_domains = copy.deepcopy(domains)
    deletions: List[Tuple[Var, Color]] = []

    new_domains[var] = [val]

    for n in adj[var]:
        if len(new_domains.get(n, [])) == 1 and n != var:
            continue
        if n in new_domains and val in new_domains[n]:
            new_domains[n].remove(val)
            deletions.append((n, val))
            if len(new_domains[n]) == 0:
                return new_domains, False, deletions

    return new_domains, True, deletions


def is_consistent(var: Var, val: Color, assignment: Dict[Var, Color], adj: Adj) -> bool:
    for n in adj[var]:
        if n in assignment and assignment[n] == val:
            return False
    return True


def bt_fc(
    assignment: Dict[Var, Color],
    domains: Domains,
    adj: Adj,
    log: List[str],
) -> Optional[Dict[Var, Color]]:
    if all(len(domains[v]) == 1 for v in domains):
        final = {v: domains[v][0] for v in domains}
        return final

    var = select_var_MRV_degree(assignment, domains, adj)
    for val in order_values_LCV(var, domains, adj):
        if not is_consistent(var, val, assignment, adj):
            continue

        log.append(f"[Choose] {var}={val}")
        new_domains, ok, deletions = forward_check(var, val, domains, adj)

        def fmt_domains(D: Domains) -> str:
            keys = sorted(D.keys())
            parts = [f"{k}:{D[k]}" for k in keys]
            return "{" + ", ".join(parts) + "}"

        log.append(f"  [FC] After assigning {var}={val}: {fmt_domains(new_domains)}")
        if not ok:
            log.append(f"  [FC] Domain wipe-out -> backtrack on {var}={val}")
            continue

        assignment[var] = val
        res = bt_fc(assignment, new_domains, adj, log)
        if res is not None:
            return res
        # откат
        assignment.pop(var, None)
        log.append(f"[Backtrack] undo {var}={val}")

    return None


if __name__ == "__main__":
    variables, domains, adj = csp_topology()
    log: List[str] = []

    solution = bt_fc(assignment={}, domains=domains, adj=adj, log=log)

    print("=== CSP Backtracking + Forward Checking Trace ===")
    for line in log:
        print(line)

    if solution:
        print("\nSolution:")
        for k in sorted(solution.keys()):
            print(f"  {k} = {solution[k]}")
    else:
        print("\nNo solution found.")


=== CSP Backtracking + Forward Checking Trace ===
[Choose] R1=Blue
  [FC] After assigning R1=Blue: {R1:['Blue'], R2:['Red', 'Green'], R3:['Red', 'Green'], R4:['Red', 'Green']}
[Choose] R3=Green
  [FC] After assigning R3=Green: {R1:['Blue'], R2:['Red'], R3:['Green'], R4:['Red']}

Solution:
  R1 = Blue
  R2 = Red
  R3 = Green
  R4 = Red
